# Notebook 2: Satellite Data Aggregation
Replication code for:
- Figure S2 Panel A
- Figure S8

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from matplotlib.colors import LinearSegmentedColormap, Normalize, LogNorm
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes,mark_inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from scipy.spatial import cKDTree
from shapely.geometry import Point

In [ ]:
cantons = gpd.read_file('data/shapefiles/cantons.geojson')
cantons['nation'] = 'TGO'
print('Number of cantons: %i' % len(cantons))
nation = cantons.dissolve(by='nation')

In [ ]:
pop = pd.read_csv('data/satellite/pop.csv')
pop = gpd.GeoDataFrame(pop, 
                       geometry=gpd.points_from_xy(pop['Lon'], pop['Lat']))
pop.crs={'init': 'epsg:4326'}

In [ ]:
rwi = pd.read_csv('data/satellite/wealth.csv')
rwi = gpd.GeoDataFrame(rwi, 
                       geometry=gpd.points_from_xy(rwi['longitude'], rwi['latitude']))
rwi.crs={'init': 'epsg:4326'}

In [ ]:
sample = pop.sample(10000)
sample = gpd.sjoin(sample, nation, op='within')

In [ ]:
ax = nation.plot(color='lightgrey', figsize=(10, 10))

ax = rwi.plot(markersize=1, marker='s', column=rwi['rwi'], cmap='magma', 
         legend=True, legend_kwds={'shrink':0.5, 'label':'Relative Wealth Index (RWI)'}, 
         ax=ax)

ax.set_title('Wealth Estimates', fontsize='xx-large')

ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax.axis('off')

plt.show()

In [ ]:
ax = nation.plot(color='lightgrey', figsize=(10, 10))

ax = sample.plot(ax=ax, color='mediumblue', 
                 markersize=sample['Population']/10000)

ax.set_title('Population Density Estimates', fontsize='xx-large')

ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax.axis('off')

plt.show()

In [ ]:
def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

In [ ]:
overlay = ckdnearest(pop, rwi)
overlay.columns = ['longitude_pop', 'latitude_pop', 'pop', 'geometry', 
                   'latitude_rwi', 'longitude_rwi', 'rwi', 'error', 'dist']
overlay = overlay.groupby(['latitude_rwi', 'longitude_rwi', 'rwi'], as_index=False)\
    .agg('sum')
overlay = overlay[['latitude_rwi', 'longitude_rwi', 'pop', 'rwi']]\
    .rename({'pop':'pop_weight'}, axis=1)
overlay['weighted_rwi'] = overlay['pop_weight']*overlay['rwi']
overlay = gpd.GeoDataFrame(overlay, geometry=gpd.points_from_xy(overlay['longitude_rwi'], 
                                                                overlay['latitude_rwi']))

overlay.crs={'init': 'epsg:4326'}

In [ ]:
maps = {}
for name, numberkey, df in [('cantons', 'canton', cantons)]:
    
    grouped = gpd.sjoin(overlay, df, op='within', how='right')
    grouped = grouped.groupby(numberkey, as_index=False).agg('sum')
    grouped['rwi'] = grouped['weighted_rwi']/grouped['pop_weight']
    grouped = grouped[[numberkey, 'rwi', 'pop_weight']]\
        .rename({'pop_weight':'pop'}, axis=1)
    grouped = df[[numberkey, 'geometry']].merge(grouped, how='left', 
                                                         on=numberkey)
    
    nulls = grouped[pd.isnull(grouped['rwi'])].copy()
    nonnulls = grouped[~pd.isnull(grouped['rwi'])].copy()
    
    imputed = []
    for index, row in nulls.iterrows():  
        neighbors = nonnulls[nonnulls.geometry.touches(row['geometry'])]
        imputed.append(neighbors['rwi'].mean())
    nulls['rwi'] = imputed
    
    grouped = pd.concat([nulls, nonnulls])
    maps[name] = grouped

    

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))

nation.plot(ax=ax, color='lightgrey')
ax.axis('off')
    
maps['cantons'].plot(ax=ax, column='rwi', cmap='magma', legend=True, 
                       legend_kwds={'shrink':0.5, 'label':'Relative Wealth Index (RWI)'})

ax.set_title('Wealth by Canton', fontsize='xx-large')

plt.show()